# Introduction

This is the first of two notebooks related to the Applied Data Science Capstone project. This notebook contains a short presentation of the problem, the methodology as well as all steps that are required to collect the data. The second notebook contains all the data exploration, modelling and evaluation steps.

# Problem definition

In this project we set out to build a model that predicts the number of bike shops in a zip code area in New York City. We'll use this information to give a recommendation on where to open a bike shop. This recommendation is based on the difference between existing and predicted number of bike shops. Our model leverages foursquare data, data from the American Community Census as well as other data sources. 


# Data collecting

We start by importing the necessary libraries.

In [18]:
# Import necessary libraries
import pandas as pd
import numpy as np
import folium
import json, requests
from uszipcode import SearchEngine
import ast
import itertools
from itertools import chain, combinations


# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Evaluation and models
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
import scipy
import statistics

We start by downloading a geojson file that contains all zip codes in the state of New York. This provides us with a complete list of zip codes as well as a possibility to visualize results on a folium map.

In [19]:
#Download the GeoJSON file
!wget document --output-document=ny_geo.json -nc https://github.com/OpenDataDE/State-zip-code-GeoJSON/raw/master/ny_new_york_zip_codes_geo.min.json

Die Datei »ny_geo.json« ist schon vorhanden; kein erneuter Download.


After extracting a list of all zip codes in the state of New York we use the uszipcode library to filter out those zip codes that belong to New York City and at latitude and longitude information. We find 211 zip codes

In [20]:
# Extract all postal codes that belong to the NYC area
# Open the file
ny_geojson_filename = 'ny_geo.json'
with open(ny_geojson_filename) as f:
    data = json.load(f)

# Build the basic dataframe and define NYC counties
ny_df = pd.DataFrame(columns = ['Zip Code', 'County', 'Lat', 'Long', 'Area'])
nyc_counties_set = ('New York County', 'Bronx County', 'Queens County', 'Kings County', 'Richmond County')

# Get a search object from the uszipcode library
search = SearchEngine(simple_zipcode = False)

# Loop over the geojson 
for r in data['features']:
    
    # Use the search object to get information related to the zip codes
    a = search.by_zipcode(r['properties']['ZCTA5CE10'])
    
    # Get a list from the returned object
    a = a.values()
    
    # If the zip code belongs to NYC, add all the information to the dataframe
    if a[4] in nyc_counties_set:
        ny_df = ny_df.append({'Zip Code':r['properties']['ZCTA5CE10'], 'County': a[4],
                              'Lat': a[6], 'Long': a[7], 'Area': a[13]}, ignore_index = True)

In [21]:
print("Rows in the dataframe: " + str(max(ny_df.shape)))
ny_df.head()

Rows in the dataframe: 211


,Zip Code,County,Lat,Long,Area
0,10464,Bronx County,40.86,-73.79,3.50
1,10470,Bronx County,40.90,-73.86,1.42
2,10455,Bronx County,40.81,-73.91,0.71
3,10473,Bronx County,40.82,-73.86,2.18
4,11234,Kings County,40.61,-73.89,7.55


## Foursquare API calls

The next cells contain all tasks required to extract a list of unique bike shops and relate them to their respective zip codes. The first of the two functions below is used to build a API-call url based on a set of keywords.
The second function returns a list of unique locations. We need this second function because the foursquare API does not directly allow us to specify a zip code area, but rather uses latitude and longitude information. We therefore call the foursquare api using the zip code latitude and longitude. After that, we map the results to a zip code area and discard any duplicate entries.

In [22]:
# This cell defines two functions that we use to simplify the Foursquare API calls

def foursquare_url(endpoint, **kwargs):
    # This function constructs a url that can be used to make an call to the Foursquare API.
    
    # Foursquare Login Credentials
    foursquare_id_client = ''
    foursquare_id_secret = ''
    foursquare_id_version = ''   
    
    # Construct base url with selected api endpoint and authentication credentials
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
        endpoint, foursquare_id_client, foursquare_id_secret, foursquare_id_version) 

    if endpoint == 'search':
        # Search Keywords: 'radius', 'query', 'limit', 'latitude', 'longitude', 'near', 'categoryId'
        coordinates_flag = False
        near_flag = False
        for key, value in kwargs.items():
            # Handle simple keys: radius, query and limit
            if key in {'radius', 'query', 'limit'}:
                url = url + '&{}={}'.format(key, value)
            # Handle coordinate keys: If one of either is found 
            # and no coordinates have yet been added
            elif (key == 'latitude' or key == 'longitude'):
                # check if no near information has been provided
                if not near_flag and not coordinates_flag:
                    # Check if arguments contain both lat and long information, then add them to
                    # url and set the coordinate flag to True
                    if 'latitude' in kwargs and 'longitude' in kwargs:
                        url = url + '&ll={},{}'.format(kwargs.get('latitude'), kwargs.get('longitude'))
                        coordinates_flag = True
                    # Raise exception if either keyword is corrupted or missing
                    else:
                        print('Error with entering coordinates')
                        return
                # Raise exception if keywords contain both coordinates and near information
                elif near_flag:
                    print('Coordinates and near a location used simultaneously.')
                    return
            # Handle near key: If a near key has been provided and no coordinates were entered
            elif key == 'near':
                # Check if no coordinates were entered yet, then add near parameter to url
                # and set near_flag to true 
                if not coordinates_flag:   
                    url = url + '&{}={}'.format(key, value)
                    near_flag = True
                # Raise exception if keywords contain both coordinates and near information
                else:
                    print('Coordinates and near a location used simultaneously.')
                    return
            elif key == 'categoryId':
                Id_string = ",".join(value)
                url = url + '&{}={}'.format(key, Id_string)
            else:
                print('Wrong key: ' + key)
                return
    return url

def search_foursquare(location_df,idlist):
    # This function is used to build a dictionairy containing unique locations. It takes a dataframe with latitude
    # and longitude information as well as a list of desired category ids and returns a dict that contains
    # unique venues, with the key being the venue id and the value being the rest of the information returned
    # by the foursquare API. 
    
    # Initialize empty dict
    results_dict = {}
    # Loop over the different ids
    for id in idlist:
        #Loop over the dataframe
        for index, row in location_df.iterrows():
            # Use the above function to construct the url
            search_url = foursquare_url('search', latitude = row['Lat'], longitude = row['Long'], 
                                   radius = 2000, categoryId = [id])
            # Make API call
            response = requests.get(search_url).json()
            # Check if call returned any venues
            if response['response']['venues']:
                # Loop over result list
                for result in response['response']['venues']:
                    # Check if venue has already been added to the dict
                    if result['id'] not in results_dict:
                        # Add entry to result_dict in the form of {'id':{key:value for all other information}}
                        results_dict[result['id']] = {i:result[i] for i in result if i!='id'}
    # Return the resulting dict
    return results_dict

In [23]:
# Call the above functions to get a dict with all bike shops in NYC
bikeshops_dict = search_foursquare(ny_df, ['4bf58dd8d48988d115951735'])

Using the method described above we find 370 unique bike shops.

In [24]:
# We find 370 unique bike shops
print("Number of unique bike shops: " + str(len(bikeshops_dict)))
bikeshops_dict

Number of unique bike shops: 373


{'4bd7372b0b779c74f8ad04a0': {'name': 'County Cycle Center',
  'location': {'address': '970 McLean Ave',
   'crossStreet': 'Martha Avenue',
   'lat': 40.903306,
   'lng': -73.865639,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.903306,
     'lng': -73.865639},
    {'label': 'entrance', 'lat': 40.903177, 'lng': -73.865683}],
   'distance': 600,
   'postalCode': '10704',
   'cc': 'US',
   'city': 'Yonkers',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['970 McLean Ave (Martha Avenue)',
    'Yonkers, NY 10704',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d115951735',
    'name': 'Bike Shop',
    'pluralName': 'Bike Shops',
    'shortName': 'Bike Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bikeshop_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1598718037',
  'hasPerk': False},
 '50608454e4b0fc173099b6d5': {'name': 'Bronx River Bicycle Works',
  'location': {'address': '27 Mount 

We now build a dataframe that contains all necessary information to count the number of bike shops in a zip code area. 

In [25]:
# We now construct a dataframe that contains all necesarry information to assign bike shops to zip code areas
bikeshop_df = pd.DataFrame(columns = ['Id', 'Name', 'Zip Code', 'Lat', 'Long'])

# Function to check if the dict entry contains zip code information
def check_pc(location):
    if 'postalCode' in location:
        return location['postalCode']
    else:
        return "Error"
# Build a dataframe from the dict
for i in bikeshops_dict:
    bikeshop_df = bikeshop_df.append({'Id' : i, 'Zip Code': check_pc(bikeshops_dict[i]['location']), 
                                      'Name': bikeshops_dict[i]['name'],
                                      'Lat' : bikeshops_dict[i]['location']['lat'],
                                      'Long' : bikeshops_dict[i]['location']['lng']}, ignore_index=True)

In [26]:
bikeshop_df.head()

,Id,Name,Zip Code,Lat,Long
0,4bd7372b0b779c74f8ad04a0,County Cycle Center,10704,40.903306,-73.865639
1,50608454e4b0fc173099b6d5,Bronx River Bicycle Works,10550,40.911275,-73.845048
2,4bf5525594af2d7fb78c3b72,Kim's Motorcycle Corp,10704,40.905510,-73.870253
3,511fd14be4b0b9b40101898a,bike shop,10466,40.889973,-73.859406
4,4d60403aef378cfaf9147ba6,Arrow Cycle Inc,10466,40.890157,-73.859414


Some entries do not contain zip code information. We resort to the uszipcode library to map a zip code to these bike shops based on their coordinates.

In [27]:
# Use the uszipcode library to find missing postal codes

# Iterate over the bikeshop dataframe
for index, row in bikeshop_df.loc[bikeshop_df['Zip Code'] == 'Error'].iterrows():
    a = search.by_coordinates(row['Lat'], row['Long'], returns = 1)
    a = a[0].to_dict()
    bikeshop_df.loc[index, 'Zip Code'] = a['zipcode']

We can now count the number of bike shops in each zip code area.

In [28]:
# Build a Dataframe that shows the number of bike shops in each zip code area
bikeshop_count_df = bikeshop_df.groupby(['Zip Code'])[['Name']].count()
bikeshop_count_df.reset_index(inplace = True)
bikeshop_count_df.rename(columns = {'Name':'Bike Shops'}, inplace = True)

In [29]:
print("Number of zip code areas that have a bike shop: " + 
     str(bikeshop_count_df['Zip Code'].count()))
print("Total number of bike shops: " + str(bikeshop_count_df['Bike Shops'].sum()))
bikeshop_count_df.sort_values(by = 'Bike Shops', ascending = False).head(10)


Number of zip code areas that have a bike shop: 123
Total number of bike shops: 373


,Zip Code,Bike Shops
6,10002,16
74,11211,16
69,11206,12
16,10019,10
12,10013,9
78,11215,9
85,11222,8
9,10009,8
32,10036,8
83,11220,7


We can now merge this resulting dataframe with the original dataframe

In [30]:
# Merge the number of bikeshops in each zip code area with the original dataframe
ny_df2 = ny_df.merge(bikeshop_count_df, on = 'Zip Code', how = 'left')

# Replace all missing values with 0
ny_df2['Bike Shops'].fillna(value = 0, inplace = True)

# Change data type of bike shops column
ny_df2[['Bike Shops']] = ny_df2[['Bike Shops']].astype('int64')

# Show top ten zip code areas
ny_df2.sort_values(by = 'Bike Shops', ascending = False).head(10)

,Zip Code,County,Lat,Long,Area,Bike Shops
60,10002,New York County,40.72,-73.99,0.88,16
81,11211,Kings County,40.71,-73.95,2.30,16
31,11206,Kings County,40.70,-73.94,1.43,12
168,10019,New York County,40.77,-73.99,0.68,10
6,11215,Kings County,40.67,-73.98,2.18,9
112,10013,New York County,40.72,-74.01,0.55,9
89,10036,New York County,40.76,-73.99,0.44,8
11,10009,New York County,40.73,-73.98,0.62,8
107,11222,Kings County,40.73,-73.95,1.52,8
83,11220,Kings County,40.64,-74.02,1.79,7


In [31]:
# Print the number of zip code areas in NYC that have a bike shop
print("Number of zip code areas that have a bike shop: " + 
     str(ny_df2['Zip Code'].loc[ny_df2['Bike Shops'] != 0].count()))

# Print the total number of bike shops in the combined dataframe
print("Total number of bike shops: " + str(ny_df2['Bike Shops'].sum()))

Number of zip code areas that have a bike shop: 110
Total number of bike shops: 347


It is noteworthy that the above numbers are smaller after merging the dataframe that counted the number of bike shops with our original dataframe. This is because our foursquare api call results (which were based on cordinates and a specified radius) included bike shops located outside of NYC. After merging, these entries dissappear.

## ACS API calls

The next two cells are used to get demographic and economic data for all NYC zip code areas.

In [32]:
# Get Census Data from the ACS:
# - Population: Inhabitants for each zip code area
# - Income: Average income in each zip code area 
# - Bike to work: Number of inhabitants that use a bike to get to work

# Define API key and dataframe
acs_key = ''
ny_acs_df = pd.DataFrame(columns = ['Zip Code', 'Population', 'Income', 'Bike to work'])

# Loop over all zip codes
for index, row in ny_df2.iterrows():
    # API URL. Keywords are B01003_001E, B19013_001E and B08006_014E
    acs_url = ('https://api.census.gov/data/2018/acs/acs5?' +
    'get=B01003_001E,B19013_001E,B08006_014E&for=zip%20code%20tabulation%20area:' + row['Zip Code']) + '&key=' + acs_key
    
    # Make API call
    acs_page = requests.get(acs_url, allow_redirects=False, timeout=10)
    acs_page = acs_page.text
    # The result page is a string in the form of a nested list.
    # Use the ast extension to turn the string into an actual list
    try:
        acs_result = ast.literal_eval(acs_page)
    # Handle errors    
    except SyntaxError:
        print(row['Zip Code'] + ':' + acs_page + '\n')
        acs_b_result = [['empty'],['unknown','unknown', 'unknown']]
            
    # Extract relevant variables from result list
    population_acs = acs_result[1][0]
    income_acs = acs_result[1][1]
    biketowork_acs = acs_result[1][2]
    
    # Append the dataframe
    ny_acs_df = ny_acs_df.append({'Zip Code':row['Zip Code'], 'Population' : population_acs,
                                  'Income':income_acs, 'Bike to work': biketowork_acs}, ignore_index = True)

In [33]:
# Get Business Data from the ACS:
# Payroll: Total salaries paid in a zip code
# Employees: Number of employees in a zip code


# Define API key and dataframe
acs_key = ''
ny_acs_b_df = pd.DataFrame(columns = ['Zip Code', 'Employees', 'Payroll'])

# Loop over all zip codes
for index, row in ny_df2.iterrows():
    # API URL. Keywords are EMP and PAYANN
    acs_b_url = 'https://api.census.gov/data/2018/zbp?get=EMP,PAYANN&for=zipcode:' + row['Zip Code'] + '&key=' + acs_key
    
    # Make API call
    acs_b_page = requests.get(acs_b_url, allow_redirects=False, timeout=10)
    acs_b_page = acs_b_page.text
    # The result page is a string in the form of a nested list.
    # Use the ast extension to turn the string into an actual list
    try:
        acs_b_result = ast.literal_eval(acs_b_page)
    # Handle errors  
    except SyntaxError:
        print(row['Zip Code'] + ':' + acs_b_page + '\n')
        acs_b_result = [['empty'],['unknown','unknown']]
        
    # Extract relevant variables from result list
    employees_acs = acs_b_result[1][0]
    payroll_acs = acs_b_result[1][1]
    
    # Append dataframe
    ny_acs_b_df = ny_acs_b_df.append({'Zip Code':row['Zip Code'], 'Employees':employees_acs,
                                      'Payroll':payroll_acs}, ignore_index = True)

11424:

11425:



In [34]:
# Summary of errors
print(ny_acs_b_df.loc[ny_acs_b_df['Payroll'] == 'unknown'])
print(ny_acs_b_df.loc[ny_acs_df['Population'] == 'unknown'])

   Zip Code Employees  Payroll
26    11424   unknown  unknown
84    11425   unknown  unknown
Empty DataFrame
Columns: [Zip Code, Employees, Payroll]
Index: []


In [35]:
# Safe all dataframes
ny_acs_df.to_csv('ny_acs')
ny_acs_b_df.to_csv('ny_acs_b')
ny_df2.to_csv('ny_df2')

This cell indicates the end of all API calls. We save the resulting dataframes. The data exploration, modelling and evaluation steps are shown in the second notebook.